<a href="https://colab.research.google.com/github/zdwhite/Thinkful-Unit-2/blob/master/Thinkful_NLP_lesson_4_2_2_Run_time_demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instructions


1.  Run the first code cell.
2.  Then click runtime at the top and completely restart the runtime and skip to the second code block and continue execurint codeblocks as instructed



In [1]:
!pip install nltk
!pip install spacy

    100% |████████████████████████████████| 13.9MB 2.7MB/s 
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6


**REMINDER TO RESTART RUNTIME BEFORE RUNNING THE NEXT CODE BLOCK AND NOT AFTER (Restart and NOT Reset)**

In [1]:
!python -m spacy download en

    100% |████████████████████████████████| 37.4MB 36.2MB/s 
  Running setup.py install for en-core-web-sm ... - \ done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [2]:
import nltk
# Launch the installer to download "gutenberg" and "stop words" corpora.
nltk.download('gutenberg')
nltk.download('stopwords')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [4]:
# Import the data we just downloaded and installed.
from nltk.corpus import gutenberg, stopwords

# Grab and process the raw data.
print(gutenberg.fileids())

persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [0]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

### Timing the code on a host VM runtime

Below is a code block that imports a module to time and get a sense of how long it's going to take to run certain functions

**EVERYTHING ABOVE THIS SECTION SHOULD BE EXECUTED AFTER THE RUNTIME HAS BEEN RESTARTED**

In [0]:
import timeit



In [7]:
import spacy
nlp = spacy.load('en')
start = timeit.default_timer()

# All the processing work is done here, so it may take a while.
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  33.151894218999985


This part isn't TOO bad, for me it's ~30 -40 seconds of runtime. Nothing to really worry about.

In [8]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [0]:
 #Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    start = timeit.default_timer()
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            stop = timeit.default_timer()
            time = (stop - start)/60
            print("Processing row {}".format(i))
            print("\n Current runtime to process row {} is {} in minutes".format(i,time))
    stop = timeit.default_timer()
    total = (stop - start)/60
    return df, print(total)

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

## The real hog.

Above is an example of a function that is going to take a LONG time to run. We know this just by looking at the structure



>1.    Some variables are initialized, nothing really to see here.
2.   Then we loop over every sentence with in the book of Alice in wonderland.
3.    With in each sentence 2 opperations are checked, then yet another loop is nested where by the word(token in this case) is checked against a list of common words.
4.    This means that there is 3 nested iterables which means that as the size of X (X being the length of data that increases the run time increases by the power of 3 or X^3)
5.    Lastly a second nested loop checks each word and the number of times it appears with in the list of words (this loop is of neglable influence on runtime as X becomes larger)
6.    What we should observe when invoking this function, is that for relatively small lengths of X their run times for every 50 rows should be exponentially shorter than large lenths of X. **I've added a way to test this below**


In [18]:
#First let's run 200 rows of data which will produce 3 splits and a total run time
x = 200
test = sentences.iloc[:x,:]
#test = sentences
#start = timeit.default_timer()

# Create our data frame with features. This can take a while to run.
word_counts = bow_features(test, common_words)


Processing row 0

 Current runtime to process row 0 is 0.0036449761000009553 in minutes
Processing row 50

 Current runtime to process row 50 is 0.03625695796666643 in minutes
Processing row 100

 Current runtime to process row 100 is 0.08301622416666797 in minutes
Processing row 150

 Current runtime to process row 150 is 0.11577711826666738 in minutes
0.14843455413333306


In [23]:
#0.14843455413333306
#0.14843455413333306/(200/len(sentences))

200/500

0.4

We can see that the total runtime here is not even 1 minute and we pushed through ~1/25 or less than 5% of our total data. 
>**If our function's runtime was linear we could predict relatively easily the runtime **

Below we will compare the same split runtime when calling the function over a data set that is only 2.5 times the size.

In [22]:
### Now lets Let's run 500 of our 5000+ rows of data or ~ 1/10th of our total data
x = 500
test = sentences.iloc[:x,:]
#test = sentences
#start = timeit.default_timer()

# Create our data frame with features. This can take a while to run.
word_counts = bow_features(test, common_words)
#word_counts.head()

Processing row 0

 Current runtime to process row 0 is 0.01685904453333175 in minutes
Processing row 50

 Current runtime to process row 50 is 0.29471184491666613 in minutes
Processing row 100

 Current runtime to process row 100 is 0.7188084216499988 in minutes
Processing row 150

 Current runtime to process row 150 is 1.030362256766663 in minutes
Processing row 200

 Current runtime to process row 200 is 1.3016407786999973 in minutes
Processing row 250

 Current runtime to process row 250 is 1.5950913646999993 in minutes
Processing row 300

 Current runtime to process row 300 is 1.9154713202833327 in minutes
Processing row 350

 Current runtime to process row 350 is 2.1680122456833297 in minutes
Processing row 400

 Current runtime to process row 400 is 2.4808673214166634 in minutes
Processing row 450

 Current runtime to process row 450 is 2.638271389266667 in minutes
2.9994781300666657


**Let's compare the Splits** (Keep in mind that this was the runtime on MY VM it may varry slightly for your VM when the code is executed else where)

The first set of data was 200 rows long and took 0.14843455413333306 (minutes) to complete.

The second set of data was 500 rows long (2.5 times as large) and took 1.3016407786999973 (minutes) to process the same amount of data. The computational runtime increased by a factor of **8.77**.



It's clear to see that this is going to take an EXTREMELY long time to run.

In [0]:
### Plug in any value for X up to the len(sentences) and compare the splits
x = 1000
test = sentences.iloc[:x,:]
#test = sentences
#start = timeit.default_timer()

# Create our data frame with features. This can take a while to run.
word_counts = bow_features(test, common_words)
#word_counts.head()